In [ ]:
!nvidia-smi

Tue May 11 01:53:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Unzip

In [ ]:
!unzip /content/drive/MyDrive/Trash_obj/data_original.zip -d /content  # images

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/batch_02_vt/0672.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0672.jpg  
  inflating: /content/batch_02_vt/0699.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0699.jpg  
  inflating: /content/batch_02_vt/1587.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._1587.jpg  
  inflating: /content/batch_02_vt/0841.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0841.jpg  
  inflating: /content/batch_02_vt/1593.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._1593.jpg  
  inflating: /content/batch_02_vt/0855.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0855.jpg  
  inflating: /content/batch_02_vt/0869.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0869.jpg  
  inflating: /content/batch_02_vt/0868.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0868.jpg  
  inflating: /content/batch_02_vt/0854.jpg  
  inflating: /content/__MACOSX/batch_02_vt/._0854.jpg  
  inflating: /content/batch_02_vt/1592.jpg  
  inflatin

# cd

In [ ]:
%cd /content/drive/MyDrive/Trash_obj/mmdetection_trash/

/content/drive/MyDrive/Trash_obj/mmdetection_trash


# Install Requierd Packages

In [ ]:
import torch
torch.__version__

'1.8.1+cu101'

In [ ]:
!pip install bottleneck-transformer-pytorch
!pip install timm
!pip install albumentations==0.4.6
!pip install adamp
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.8.1/index.html
!pip install mmdet

     |████████████████████████████████| 296kB 15.1MB/s 
     |████████████████████████████████| 122kB 14.3MB/s 
     |████████████████████████████████| 952kB 22.3MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=5549edf23499980e34ae098a071ca0847423981936d01060de337c58d2bc8134
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
  Created wheel for adamp: filename=adamp-0.3.0-cp37-none-any.whl size=5999 sha256=793615d7c2a0a9f154a9f13731f86984b9e34c172bc0384e585405a740d418bc
  Stored in directory: /root/.cache/pip/wheels/6a/89/67/879fe55977ebcbfaa5b929eb111af7fe11eb3552867850dd76
Succ

# BaseLine Code

In [ ]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')

# Data 저장해준 경로 지정
PREFIX = '/content/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
# Weight 및 Submission.csv 저장경로 지정
cfg.work_dir = './work_dirs/detectors_cascade_rcnn_r50_1x_coco'

# DetectoRS의 경우 BBox_head가 3개여서 각각을 지정, 자세한건 configs/_base_/models/cascade_rcnn_r50_fpn.py 참조
cfg.model.roi_head.bbox_head[0]['num_classes'] = 11
cfg.model.roi_head.bbox_head[1]['num_classes'] = 11
cfg.model.roi_head.bbox_head[2]['num_classes'] = 11

cfg.runner.max_epochs = 40

# Optimizer set (Default = SGD)
cfg.optimizer = dict(type='Adam', lr=0.001, weight_decay=0.0001)

# Grad Clip
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

# Model, Dataset 생성

In [ ]:
model = build_detector(cfg.model)

2021-05-11 06:56:17,893 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 06:56:17,894 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 06:56:18,405 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

In [ ]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=5.67s)
creating index...
index created!


# Train

In [ ]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-11 06:56:28,739 - mmdet - INFO - Start running, host: root@5bd1486290d5, work_dir: /content/drive/MyDrive/Trash_obj/mmdetection_trash/work_dirs/detectors_cascade_rcnn_r50_1x_coco
2021-05-11 06:56:28,740 - mmdet - INFO - workflow: [('train', 1)], max: 40 epochs


Done (t=1.22s)
creating index...
index created!


2021-05-11 06:57:02,877 - mmdet - INFO - Epoch [1][50/655]	lr: 9.890e-05, eta: 4:57:26, time: 0.682, data_time: 0.054, memory: 11587, loss_rpn_cls: 0.4093, loss_rpn_bbox: 0.0692, s0.loss_cls: 0.7748, s0.acc: 86.4531, s0.loss_bbox: 0.1769, s1.loss_cls: 0.3320, s1.acc: 85.4775, s1.loss_bbox: 0.0484, s2.loss_cls: 0.1387, s2.acc: 89.3164, s2.loss_bbox: 0.0065, loss: 1.9557, grad_norm: 13.6972
2021-05-11 06:57:34,787 - mmdet - INFO - Epoch [1][100/655]	lr: 1.988e-04, eta: 4:47:14, time: 0.638, data_time: 0.008, memory: 11587, loss_rpn_cls: 0.1888, loss_rpn_bbox: 0.0637, s0.loss_cls: 0.5048, s0.acc: 88.5234, s0.loss_bbox: 0.2690, s1.loss_cls: 0.1426, s1.acc: 94.8545, s1.loss_bbox: 0.0852, s2.loss_cls: 0.0415, s2.acc: 97.5166, s2.loss_bbox: 0.0126, loss: 1.3082, grad_norm: 6.2122
2021-05-11 06:58:06,621 - mmdet - INFO - Epoch [1][150/655]	lr: 2.987e-04, eta: 4:43:15, time: 0.637, data_time: 0.008, memory: 11587, loss_rpn_cls: 0.1651, loss_rpn_bbox: 0.0618, s0.loss_cls: 0.4589, s0.acc: 89.2295

# Inference

In [ ]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [ ]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/detectors_cascade_rcnn_r50_1x_coco.py')

# Data 저장해준 경로 지정
PREFIX = '/content/'

# Inference해줄 특정 Epoch의 Weight를 가져오기
epoch = 30

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
# Weight 및 Submission.csv 저장경로 지정
cfg.work_dir = './work_dirs/detectors_cascade_rcnn_r50_1x_coco'

# DetectoRS의 경우 BBox_head가 3개여서 각각을 지정, 자세한건 configs/_base_/models/cascade_rcnn_r50_fpn.py 참조
cfg.model.roi_head.bbox_head[0]['num_classes'] = 11
cfg.model.roi_head.bbox_head[1]['num_classes'] = 11
cfg.model.roi_head.bbox_head[2]['num_classes'] = 11

# Grad Clip
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

In [ ]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [ ]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-11 06:45:32,900 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-11 06:45:32,902 - mmdet - INFO - Use load_from_torchvision loader
2021-05-11 06:45:33,421 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

Use load_from_local loader


In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 9.9 task/s, elapsed: 85s, ETA:     0s

# Submission.csv 생성

In [ ]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names

# Submission File 이름 설정
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.68147224 159.63731 58.50815 255.68462 170....,batch_01_vt/0021.jpg
1,1 0.50998807 0.0 239.62106 306.45398 457.1768 ...,batch_01_vt/0028.jpg
2,1 0.8188487 80.50423 170.56989 366.66602 375.9...,batch_01_vt/0031.jpg
3,8 0.9698158 23.756287 205.9558 373.59396 373.1...,batch_01_vt/0032.jpg
4,1 0.9927779 389.3208 265.02182 484.4649 344.44...,batch_01_vt/0070.jpg
